In [147]:
import pandas as pd
import re
import difflib
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import requests
import os

In [148]:
#Get sheet names from Excel File
Ex_file = pd.ExcelFile('MyPoert AUG sheet1.xlsx')
sheet_names = Ex_file.sheet_names

In [149]:
df_myport = pd.DataFrame()

for sheet in sheet_names:
    df = pd.read_excel('MyPoert AUG sheet1.xlsx',sheet_name=sheet)
    
    df = df[df.columns[0:2]]
    df = df.rename(columns={df.columns[0]: 'trip', df.columns[1]: 'details'})
    df['Country'] = sheet
    
    df_myport = pd.concat([df_myport,df])


country = df_myport.pop('Country')
df_myport.insert(0,'Country',country)
df_myport.reset_index(drop=True,inplace=True)

df_myport.ffill(inplace=True)
df_myport.drop_duplicates(inplace=True)

index_of_duplicated = list(df_myport[(df_myport[['Country','trip']].duplicated())].index)

for i in index_of_duplicated:
    df_myport['details'].loc[i-1] += ' \n ' + df_myport['details'].loc[i]


df_myport.drop(index=index_of_duplicated,inplace=True)

df_myport.head(2)

,Country,trip,details
0,Reunion,FROM Frederikshavn TO Liepaja,Trip: ONE WAY\nPrice starts from:EUR 3306\n
1,Reunion,from Emden TO Fos-sur-Mer,Trip:ONE WAY\nprice start from :EUR 6363


In [150]:
df_myport['trip_type'] = df_myport['details'].apply(lambda x : 'R' if 'round' in x.lower() else 'O')

df_myport['price'] = (df_myport['details'].str.findall('(\d+)').apply(lambda x : int(x[0]) if x else None))


In [151]:
def trans_to_arabic(x):
    # Define the translation table
    arabic = '٠١٢٣٤٥٦٧٨٩'
    english = '0123456789'
    translation_table = str.maketrans(english, arabic)

    # Translate English numerals to Arabic numerals
    arabic_num = str(x).translate(translation_table)
    return arabic_num  # Outputs: ١٢٣٤٥٦٧٨٩٠

df_myport['price_ar'] = df_myport['price'].apply(trans_to_arabic)

df_myport.head(2)

,Country,trip,details,trip_type,price,price_ar
0,Reunion,FROM Frederikshavn TO Liepaja,Trip: ONE WAY\nPrice starts from:EUR 3306\n,O,3306,٣٣٠٦
1,Reunion,from Emden TO Fos-sur-Mer,Trip:ONE WAY\nprice start from :EUR 6363,O,6363,٦٣٦٣


In [152]:
df_myport['from'] = df_myport['trip'].str.extract('FROM (.*)TO ', flags=re.IGNORECASE)
df_myport['to'] = df_myport['trip'].str.extract('TO (.*)', flags=re.IGNORECASE)

In [153]:
error_from = list(df_myport[df_myport['from'].isna()].index)

for i in error_from:
    df_myport.at[i,'from'] = df_myport.loc[[i]]['trip'].str.extract('FRM (.*) TO', flags=re.IGNORECASE).loc[i][0]


df_myport['from'] = df_myport['from'].apply(lambda x : str(x).strip())

In [154]:
respose = requests.get('https://geographyfieldwork.com/WorldCapitalCities.htm')
soup = BeautifulSoup(respose.text,'html.parser')

table = soup.find('table',attrs={'summary':"World Capitals"})
table_rows = table.find_all("tr")

In [155]:
row_data = []

for tr in table_rows:
    td = tr.find_all("td")
    data = [cell.text for cell in td]
    row_data.append(data)

row_data = row_data[1:-1]

In [156]:
countries = []
capitals = []

for country , capital in row_data:
    countries.append(country)
    capitals.append(capital)


In [157]:
all_countries = pd.DataFrame(data={'country':countries,'capital':capitals})

In [158]:
all_countries['capital'] = all_countries['capital'].apply(lambda x : x.split('[')[0])
all_countries['capital'] = all_countries['capital'].apply(lambda x : x.split('(')[0])

all_countries['country'] = all_countries['country'].apply(lambda x : x.split('[')[0])
all_countries['country'] = all_countries['country'].apply(lambda x : x.split('(')[0])

In [159]:
all_countries.head(2)

,country,capital
0,Afghanistan,Kabul
1,Albania,Tirana


In [160]:
country_df = pd.read_excel('Countries IDs.xlsx')
country_df.head(2)

,Country,ID
0,Reunion,23
1,Fiji,28


In [161]:
df = pd.merge(df_myport, country_df, how='outer', on='Country')

null_country = df[df['trip'].isna()].index
df.drop(null_country,inplace=True)

In [162]:
list_of_capitals =  list(df[df['ID'].isna()]['Country'].unique())
list_of_capitals

['Paramaribo', 'Isle of Man', 'Majuro']

In [163]:
for i in range(len(list_of_capitals)):
    try:
        country = list(all_countries[all_countries['capital'] == list_of_capitals[i]]['country'])[0]
        df.replace(list_of_capitals[i],country, inplace=True)
    except:
        continue

In [164]:
df = pd.merge(df , country_df, how='outer', on='Country')
null_country = df[df['trip'].isna()].index
df.drop(null_country,inplace=True)
df.head(2)

,Country,trip,details,trip_type,price,price_ar,from,to,ID_x,ID_y
0,Reunion,FROM Frederikshavn TO Liepaja,Trip: ONE WAY\nPrice starts from:EUR 3306\n,O,3306.0,٣٣٠٦,Frederikshavn,Liepaja,23.0,23.0
1,Reunion,from Emden TO Fos-sur-Mer,Trip:ONE WAY\nprice start from :EUR 6363,O,6363.0,٦٣٦٣,Emden,Fos-sur-Mer,23.0,23.0


In [165]:
df.drop(['ID_x'],axis=1,inplace=True)
df.rename(columns={'ID_y':'Country_ID'},inplace=True)
df.head(2)

,Country,trip,details,trip_type,price,price_ar,from,to,Country_ID
0,Reunion,FROM Frederikshavn TO Liepaja,Trip: ONE WAY\nPrice starts from:EUR 3306\n,O,3306.0,٣٣٠٦,Frederikshavn,Liepaja,23.0
1,Reunion,from Emden TO Fos-sur-Mer,Trip:ONE WAY\nprice start from :EUR 6363,O,6363.0,٦٣٦٣,Emden,Fos-sur-Mer,23.0


In [166]:
# like Isle of Man
country_problem = list(df[df['Country_ID'].isna()]['Country'].unique())

problem_collector = {}
for i in country_problem :
    problem_collector[i] = "No ID Founded in Countries IDs sheet"
    index = df[df['Country']== i].index
    df.drop(index , inplace = True)

In [167]:
ports_df = pd.read_excel('Ports_ceties_codes_and_IDs.xlsx')
ports_df.rename(columns={'ID':'port_ID'},inplace=True)

In [168]:
ports_df.head(2)

,City,Port Name,Code,port_ID
0,Guttenberg,A Coruña,OGX,870
1,Union City,Aalborg,FFO,229


In [169]:
def find_close_match(city, df2):
    # Use difflib to find the most similar city name in df2
    close_match = difflib.get_close_matches(city, df2, n=1)
    return close_match[0] if close_match else city

In [170]:
all_ports_and_cities = pd.concat([ports_df['Port Name'],ports_df['City']]).reset_index(drop=True)

df['from_port'] = df['from'].apply(find_close_match, args=(all_ports_and_cities,))
df['to_port'] = df['to'].apply(find_close_match, args=(all_ports_and_cities,))


In [171]:
list1 = df['from_port'].unique()
for i in list1:
    if i in ports_df['City'].to_list():
        print(i)

Perth Amboy
Berwyn


In [172]:
def handel_cities_names (column):
    list1 = df[column].unique()
    for i in list1:
        if i in ports_df['City'].to_list():

            filter = ports_df[ports_df['City'] == i]

            if len(filter) == 0 :
                problem_collector[i] = 'No mactched Ports for this City in Ports_ceties_codes_and_IDs'
                index = df[df[column]== i].index
                df.drop(index , inplace = True)

            elif len(filter) == 1 :
                df[column].replace(i,filter.values[0][1],inplace=True)

            elif len(filter) > 1 :
                problem_collector[i] = 'There are more than one port For this City in Ports_ceties_codes_and_IDs'
                index = df[df[column]== i].index
                df.drop(index , inplace = True)
                

In [173]:
# use the function to replace cities with ports
handel_cities_names('from_port')
handel_cities_names('to_port')

In [174]:
# Filter rows where 'Port Name' is in 'from_port'
# Get 'Code' and 'port_ID' columns

df['from_code'] = df['from_port'].apply(lambda x : ports_df[ports_df['Port Name'].isin([x])]['Code'].values[0])
df['from_id'] = df['from_port'].apply(lambda x : ports_df[ports_df['Port Name'].isin([x])]['port_ID'].values[0])

df['to_code'] = df['to_port'].apply(lambda x : ports_df[ports_df['Port Name'].isin([x])]['Code'].values[0])
df['to_id'] = df['to_port'].apply(lambda x : ports_df[ports_df['Port Name'].isin([x])]['port_ID'].values[0])


In [175]:
df.head(2)

,Country,trip,details,trip_type,price,price_ar,from,to,Country_ID,from_port,to_port,from_code,from_id,to_code,to_id
0,Reunion,FROM Frederikshavn TO Liepaja,Trip: ONE WAY\nPrice starts from:EUR 3306\n,O,3306.0,٣٣٠٦,Frederikshavn,Liepaja,23.0,Frederikshavn,Liepāja,CAC,372,ZZK,32
1,Reunion,from Emden TO Fos-sur-Mer,Trip:ONE WAY\nprice start from :EUR 6363,O,6363.0,٦٣٦٣,Emden,Fos-sur-Mer,23.0,Emden,Fos-sur-Mer,REV,572,KKA,248


In [176]:
offers_tamplet = pd.DataFrame()

offers_tamplet['port_form_id'] = df['from_id']
offers_tamplet['port_to_id'] = df['to_id']

offers_tamplet['trip_type'] = df['trip_type']

offers_tamplet['url'] = (df['from_code'] + "-" + df['to_code'] + "-" + df['trip_type']).str.lower()


In [177]:
today = datetime.today()

# Add 1 month and 3 days
future_date = today + relativedelta(months=1, days=3)


offers_tamplet['publish_date'] = today.strftime('%Y-%m-%d')
offers_tamplet['expir_date'] = future_date.strftime('%Y-%m-%d')


offers_tamplet.drop_duplicates(inplace=True)

offers_tamplet.head(2)

,port_form_id,port_to_id,trip_type,url,publish_date,expir_date
0,372,32,O,cac-zzk-o,2024-04-08,2024-05-11
1,572,248,O,rev-kka-o,2024-04-08,2024-05-11


In [178]:
offers_tamplet.to_excel('offers_tamplet.xlsx',index= False)

In [179]:
offer_content = pd.DataFrame()


offer_content['offer_url'] = (df['from_code'] + "-" + df['to_code'] + "-" + df['trip_type']).str.lower()
offer_content['country_id'] = df['Country_ID']

 
offer_content['price'] = df.apply(lambda row: {"en": int(row['price'])
                                                , "ar": row['price_ar']
                                                , "gr": int(row['price'])
                                                , "it": int(row['price'])
                                                , "cz": int(row['price'])
                                                , "fr": int(row['price'])
                                                , "sk": int(row['price'])}, axis=1)

offer_content.head(2)

,offer_url,country_id,price
0,cac-zzk-o,23.0,"{'en': 3306, 'ar': '٣٣٠٦', 'gr': 3306, 'it': 3..."
1,rev-kka-o,23.0,"{'en': 6363, 'ar': '٦٣٦٣', 'gr': 6363, 'it': 6..."


In [180]:
offer_content.to_excel('offer_content.xlsx',index= False)

In [181]:
if problem_collector :
    report = pd.DataFrame(list(problem_collector.items()),columns=['Case','Details'])
    report.to_excel('Report.xlsx',index= False)